In [13]:
import numpy as np 
import pandas as pd 
data = pd.read_csv('edited_data.csv')
X = data['Objectives']+'. '+ data['Description']
X = X.str.replace('<ul>', '').str.replace('<li>','').str.replace('</li>','').\
str.replace('</ul>','').str.replace('1.','').str.replace('2.','').str.replace('3.','').str.replace('4.','').str.replace('5.','').\
str.replace('(','').str.replace(')','').str.replace('\n','')
X = X.fillna('')

In [14]:
import nltk
#nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
nltk.download("stopwords")
stop_words = set(stopwords.words('english')) 
not_delete = ["but", "shan't", "wasn't", "couldn't", "didn't", "hadn't", "against", "no", "haven't", "shouldn't", "needn't", "wouldn't", "aren't", "mightn't", "won't", "isn't", "hasn't", "don't", "mustn't", "doesn't", "not"]
stop_words = [w for w in stop_words if w not in not_delete]
#print(stop_words)

porter = PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        if word not in stop_words:
            stem_sentence.append(porter.stem(word))
            stem_sentence.append(" ")
    return "".join(stem_sentence)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/penghanqiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
for i in range(0,X.count()):
    #print(type(i))
    X[i] = stemSentence(X[i])

In [24]:
#multilabel classifier: project tag %95
from sklearn.preprocessing import MultiLabelBinarizer

y2 = data['Project Tag']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')

for i in range(0, len(y2)):
    for j in range(0, len(y2[i])):
        y2[i][j] = y2[i][j].replace("Community Event ", "Community Event")\
        .replace("Conference/Panel Discussion ", "Conference/Panel Discussion")\
        .replace("Educational Material ", "Educational Material")\
        .replace("Social Media ", "Social Media").replace("Survey ", "Survey")\
        .replace("Teaching Activity ", "Teaching Activity")
    
mlb_y2 = mlb.fit_transform(y2)

#print(mlb.classes_)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, mlb_y2, test_size=0.3, random_state=52)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
#from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import ClassifierChain
# initialize classifier chains multi-label classifier


Classifiers = [
    #KNeighborsClassifier(),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    #RadiusNeighborsClassifier(),
    ExtraTreesClassifier(),
    RandomForestClassifier(),
    #MLPClassifier(),
    #RidgeClassifierCV()
    OneVsRestClassifier(SGDClassifier())
    ]

'''
for classifier in Classifiers:
    pipe = Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),\
                    ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
'''

'''
OneVsRest_classifier = Pipeline([
('vectorizer', CountVectorizer(ngram_range=(1, 2),stop_words='english',strip_accents = 'unicode')),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('clf', OneVsRestClassifier(SVC(C=1.0, kernel='linear')))])
'''


from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer()
X_train = Tfidf_vect.fit_transform(X_train)
X_test = Tfidf_vect.transform(X_test)

# Fit an ensemble of SVM classifier chains and take the average prediction of all the chains.
from sklearn.metrics import jaccard_score
chains = [ClassifierChain(SVC(C=1.0, kernel='linear'), order='random', random_state=i)
          for i in range(10)]
for chain in chains:
    chain.fit(X_train, y_train)
    
Y_pred_chains = np.array([chain.predict(X_test) for chain in
                          chains])
#chain_jaccard_scores = [jaccard_score(y_test, Y_pred_chain >= .5,
#                                      average='samples')
#                        for Y_pred_chain in Y_pred_chains]
#Y_pred_ensemble = Y_pred_chains.mode()

#ensemble_jaccard_score = jaccard_score(y_test,
#                                       Y_pred_ensemble >= .5,
#                                       average='samples')
                                       
#print('Chain_Classifier_test:'+ str(np.mean(Y_pred_ensemble == y_test)))
#print(ensemble_jaccard_score)


'''
Chain_classifier = Pipeline([
('vectorizer', CountVectorizer(ngram_range=(1, 2),stop_words='english',strip_accents = 'unicode')),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('clf', ClassifierChain(SVC(C=1.0, kernel='linear')))])
Chain_classifier.fit(X_train, y_train) 
predicted = Chain_classifier.predict(X_test)
print(Chain_classifier.score(X_test, y_test))
print('ChainClassifier_test:'+ str(np.mean(predicted == y_test)))
'''

'''
OneVsRest_classifier.fit(X_train, y_train) 
predicted = OneVsRest_classifier.predict(X_test)
print(OneVsRest_classifier.score(X_test, y_test))
print('OneVsRestClassifier_test:'+ str(np.mean(predicted == y_test)))
'''

"\nOneVsRest_classifier.fit(X_train, y_train) \npredicted = OneVsRest_classifier.predict(X_test)\nprint(OneVsRest_classifier.score(X_test, y_test))\nprint('OneVsRestClassifier_test:'+ str(np.mean(predicted == y_test)))\n"

In [37]:
from scipy import stats
a=Y_pred_chains
m = stats.mode(a)
print(m[0])

[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 1. ... 0. 0. 0.]]]


In [38]:
print('Chain_Classifier_test:'+ str(np.mean(m[0] == y_test)))

Chain_Classifier_test:0.9514660353305409


In [7]:
OneVsRest_classifier.fit(X, mlb_y2)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words='english', strip_accents='unicode',
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=Non...
                 TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True,
                                  use_idf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200,
                                                   class_weight=None, coef0=0.0,
                           

In [41]:
new_data = pd.read_csv('initial_data.csv')
X_new = new_data['Descriptions']
X_new = X_new.str.replace('\n', '').str.replace('\t','')
X_new = X_new.fillna('')
predicted1 = OneVsRest_classifier.predict(X_new)
output = mlb.inverse_transform(predicted1)

In [52]:
predicted1[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
#将predict得到的结果放到csv里面
import csv
df = pd.DataFrame(output)
df.to_csv("new_data.csv", sep = '\t')

In [3]:
sentence = 'Data science is an interdisciplinary field that uses scientific methods, processes'
a = stemSentence(sentence)
print(a)
#print(X)

data scienc interdisciplinari field use scientif method , process 


In [ ]:
print(X)

In [ ]:
#3大主要classifier：Outreach Category 85% 2选1

y = data['Outreach Category']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

'''
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(X_train, y_train) 
predicted = text_clf.predict(X_test)
print('MultinomialNB_test:'+ str(np.mean(predicted == y_test)))
'''

from sklearn.linear_model import SGDClassifier
text_clf_2 = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),('tfidf', TfidfTransformer()),('clf', SGDClassifier(alpha=0.00015)),])
text_clf_2.fit(X_train, y_train)
predicted = text_clf_2.predict(X_test)
print('SGDClassifier_test:'+ str(np.mean(predicted == y_test))) 
categories = ['Education and Public Engagement', 'Integrated Human Practices']
from sklearn import metrics
print(metrics.classification_report(y_test, predicted, target_names = categories))

'''
from sklearn.neural_network import MLPClassifier
text_clf_3 = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),\
            ('clf', MLPClassifier(solver='lbfgs', alpha=1e-5,\
            hidden_layer_sizes=(50,100,100,100,100,50,50,50), random_state=1, \
            activation = 'relu', learning_rate = 'adaptive'))])
text_clf_3.fit(X_train, y_train)
predicted = text_clf_3.predict(X_test)
print('Neural Network:'+ str(np.mean(predicted == y_test))) 
'''

In [ ]:
#multilabel classifier: project tag %95
from sklearn.preprocessing import MultiLabelBinarizer

y2 = data['Project Tag']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')

for i in range(0, len(y2)):
    for j in range(0, len(y2[i])):
        y2[i][j] = y2[i][j].replace("Community Event ", "Community Event")\
        .replace("Conference/Panel Discussion ", "Conference/Panel Discussion")\
        .replace("Educational Material ", "Educational Material")\
        .replace("Social Media ", "Social Media").replace("Survey ", "Survey")\
        .replace("Teaching Activity ", "Teaching Activity")
    
mlb_y2 = mlb.fit_transform(y2)

print(mlb.classes_)
print(mlb_y2[100:150])
print(y2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, mlb_y2, test_size=0.3, random_state=52)
#print(y_train[100:150])

In [54]:
print(str(predicted[2]))
#encode后y的样子
#print(mlb_y2)
#将标签变回string
print(mlb.inverse_transform(predicted[2:30]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[('Survey',), (), ('Consult Experts',), ('Teaching Activity',), ('Consult Experts',), (), ('Consult Experts',), ('Project Presentation',), (), ('Consult Experts',), (), (), ('Consult Experts',), ('Consult Experts',), ('Consult Experts',), ('Consult Experts', 'Survey'), ('Educational Material',), (), (), (), ('Consult Experts',), ('Consult Experts',), ('Consult Experts',), ('Consult Experts',), ('Consult Experts',), (), ('Educational Material',), ('Consult Experts',)]


In [ ]:
#将predict得到的结果放到csv里面

import csv
results = mlb.inverse_transform(predicted)
df = pd.DataFrame(results)
df.to_csv("new_data.csv", sep = '\t')

In [57]:
#测试将预测值全部变成0
p_0 = np.zeros((646,17))
print('all_zeros_test:'+ str(np.mean(p_0 == y_test)))
print(predicted.shape)

all_zeros_test:0.923511200145693
(646, 17)


In [ ]:
docs_new = ['<ul><li> Organize a "Biotech Day" and present the project to a large group of students</li><li> Address safety issues in ethics of synthetic biology</li><li> Answer their questions about synbio and the project </ul> Organized a Biotech Day for two schools to teach about iGEM, synthetic biology, and safety.', 
            'Teach high school students about synthetic biology by presenting their iGEM project during the student summer STEM outreach camp. Introduce the students to synthetic biology and answer questions and discuss with students afterwards who are interested in synthetic biology.']
predicted1 = multilabel_classifier.predict(docs_new)
for doc, category in zip(docs_new, predicted1):
    print('%r => %s' % (doc, category))

In [ ]:
#multilabel classifier: Audience %95
from sklearn.preprocessing import MultiLabelBinarizer

y2 = data['Audience']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')


a = mlb.fit_transform(y2)

#print(mlb.classes_)

In [ ]:
#测试其他classifier: Outreach Category 2选1 #NuSVC最好 82%
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),\
                    ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

In [ ]:
#算的training accuracy，之前的都是test accuracy。
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(X_train, y_train) 
predicted = text_clf.predict(X_train)
print('MultinomialNB_train:'+ str(np.mean(predicted == y_train)))

text_clf_2 = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),('tfidf', TfidfTransformer()),('clf', SGDClassifier(alpha=0.0002)),])
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neural_network import MLPClassifier
#text_clf_2 = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),('tfidf', TfidfTransformer()),('clf', MLPClassifier()),])
text_clf_2.fit(X_train, y_train)
predicted = text_clf_2.predict(X_train)
print('SGDClassifier_train:'+ str(np.mean(predicted == y_train))) 
categories = ['Education and Public Engagement', 'Integrated Human Practices']
from sklearn import metrics
print(metrics.classification_report(y_train, predicted, target_names = categories))

In [ ]:
#single sample prediction
docs_new = ['Visit Snofarm, a small dairy farm with 90 cows about an hour away from campus. Learn about the milking process from the owner of the farm. Learn about how bovine mastitis affects people in the farming industry.', 
            'Teach high school students about synthetic biology by presenting their iGEM project during the student summer STEM outreach camp. Introduce the students to synthetic biology and answer questions and discuss with students afterwards who are interested in synthetic biology.']
predicted1 = text_clf_2.predict(docs_new)
for doc, category in zip(docs_new, predicted1):
    print('%r => %s' % (doc, category))

In [ ]:
#search for best hyperparameters/parameters
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1),(1,2),(1,3)],'tfidf__use_idf': (True, False),'clf__alpha': (0.00095, 0.0009,0.0097)}
#parameters={'clf__alpha': (0.11,0.1,0.09)}
gs_clf = GridSearchCV(text_clf_2, parameters, cv=5, iid=False, n_jobs=-1)
#speed up
gs_clf = gs_clf.fit(X_train, y_train)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
#结束

In [ ]:
#载入别人train好的word embedding，估计run 15min
with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(strip_accents = 'unicode', analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])
#GloVe:太慢了1个小时都没run出来
OneVsRest_classifier = Pipeline([
("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
('clf', OneVsRestClassifier(SVC(C=1.0, kernel='linear')))])